<div style="display: flex; align-items: center; gap: 40px;">

<img src="https://framerusercontent.com/images/9vH8BcjXKRcC5OrSfkohhSyDgX0.png" width="130">


[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1OYpQFo88aLWbv0D1_DA2j4HZGRxTjRcK?usp=sharing)


<div>
  <h2>Hypothetical Document Embeddings (HyDE) RAG</h2>
  <p>HyDE operates by creating hypothetical document embeddings that represent ideal documents relevant to a given query. This method contrasts with conventional RAG systems, which typically rely on the similarity between a user's query and existing document embeddings. By generating these hypothetical embeddings, HyDE effectively guides the retrieval process towards documents that are more likely to contain pertinent information.</p>



## Get Your API Keys

Before you begin, make sure you have:

1. A SUTRA API key (Get yours at [TWO AI's SUTRA API page](https://www.two.ai/sutra/api))
2. Basic familiarity with Python and Jupyter notebooks

This notebook is designed to run in Google Colab, so no local Python installation is required.

###🔧 1. Install Required Libraries

In [20]:
!pip install -qU langchain_openai langchain_community chromadb

###🔑 2. Set Environment Variables (API Keys)

In [21]:
import os
from google.colab import userdata

# Set the API key from Colab secrets
os.environ["SUTRA_API_KEY"] = userdata.get("SUTRA_API_KEY")
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

###Initialize Openai embeddings

In [22]:
# load embedding model
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

###📄 Load and Chunk Documents

In [23]:
from langchain.document_loaders import CSVLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

loader = CSVLoader("./context.csv")
documents = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

###🔍 Vector Store: ChromaDB

In [24]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents, embeddings)

###🔁 Retriever from Vectorstore

In [25]:
retriever = vectorstore.as_retriever()

###Setup Sutra LLM (via ChatOpenAI interface)

In [31]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

# Sutra LLM setup
llm = ChatOpenAI(
    api_key=os.getenv("SUTRA_API_KEY"),
    base_url="https://api.two.ai/v2",
    model="sutra-v2"
)

# Hypothetical Answer Prompt
prompt_hyde = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that answers questions.\nQuestion: {input}\nAnswer:"),
    ("human", "{input}")
])

qa_no_context = prompt_hyde | llm | StrOutputParser()

###Hypothetical Answer Chain (HyDE style hallucination)

In [32]:
# hallucinate an answer to generate better query
question = "how does interlibrary loan work"
hypothetical_answer = qa_no_context.invoke({"input": question})
print("🧠 Hypothetical Answer:\n", hypothetical_answer)

🧠 Hypothetical Answer:
 Interlibrary loan (ILL) is a service that allows libraries to borrow books, articles, and other materials from each other on behalf of their patrons. Here’s how it typically works:

1. **Request Submission**: A patron at a library identifies a resource that is not available in their home library's collection. They can submit a request for this item through their library's ILL system, either online or in person.

2. **Verification**: The library staff verifies the request to ensure that the item is not available locally and checks its availability in other libraries within a network or consortium.

3. **Borrowing Libraries**: The home library sends the request to another library that owns the item. This could be local, regional, or even national, depending on the agreements in place.

4. **Loan Processing**: The lending library processes the request and ships the item to the requesting library. If the item is an article or a digital resource, it may be sent elect

###Retrieve Relevant Docs Based on HyDE Answer

In [33]:
# retrieve documents based on hypothetical answer
retrieval_chain = qa_no_context | retriever
retrieved_docs = retrieval_chain.invoke({"input": question})

# Combine content into string for prompt
context_text = "\n".join([doc.page_content for doc in retrieved_docs])

###Final RAG Answer Using Context

In [34]:
# RAG prompt with context
template_context = """
You are a helpful assistant that answers questions based on the provided context.
Use the provided context to answer the question.
Question: {input}
Context: {context}
"""

prompt_context = ChatPromptTemplate.from_template(template_context)
final_rag_chain = prompt_context | llm | StrOutputParser()

# Final context-aware response
final_answer = final_rag_chain.invoke({"context": context_text, "input": question})
print("✅ Final Answer:\n", final_answer)

✅ Final Answer:
 Interlibrary loan (ILL) is a service that allows patrons of one library to borrow physical materials and obtain electronic documents from another library. This service enhances access to resources that are not available in a patron's local library, making it an essential component of collection development for libraries.

The process typically involves the following steps:

1. **Request**: A patron identifies a material (book, article, etc.) that is not available at their local library and submits a request for it through the interlibrary loan service.

2. **Processing**: The local library checks its network of partner libraries to find a library that holds the requested item and is willing to lend it.

3. **Loaning**: Once a partner library agrees to lend the item, it is sent to the requesting library. This may involve shipping physical materials or providing electronic access to documents.

4. **Notification**: The local library notifies the patron when the item is a

###📊 Prepare Data for Evaluation

In [35]:
from datasets import Dataset
import pandas as pd

# Inference loop (can extend with more questions)
questions = ["how does interlibrary loan work"]
responses, contexts = [], []

for q in questions:
    docs = retriever.get_relevant_documents(q)
    ctx_text = "\n".join([doc.page_content for doc in docs])
    result = final_rag_chain.invoke({"context": ctx_text, "input": q})

    responses.append(result)
    contexts.append([doc.page_content for doc in docs])

# Create HuggingFace Dataset
data = {"query": questions, "response": responses, "context": contexts}
dataset = Dataset.from_dict(data)

# Convert to DataFrame
df = pd.DataFrame(dataset)
df

,query,response,context
0,how does interlibrary loan work,Interlibrary loan (ILL) is a service that allo...,[Procedures and methods ==\n\nAfter receiving ...
